## Composition

### Introduction

- *Composition* is another approach, different from *inheritance*, to structure code.
- Object Oriented Programming can be achieved with composition.
- Both are way to use, re-use and structure objects.

### Example: Engine Vehicle with inheritance

```python
class EngineVehicle(object):
    def __init__(self, engine_power):
        self.engine_power = engine_power
        
        
class Car(EngineVehicle):
    def __init__(self, engine_power):
        EngineVehicle.__init__(self, engine_power)
```

### Example: Engine vehicle with composition

```python
class Engine(object):
    def __init__(self, engine_power):
        self.engine_power = engine_power
        
        
class Car(object):
    def __init__(self, engine_power):
        self.engine = Engine(engine_power)
```

### Composition vs inheritance: what's the difference ?

- Inheritance defines a **is a** relationship  (or "behaves like")
- Composition defines a **has a** relationship

`(a car) is a (engine vehicle)`

`(a car) has an (engine)`


### Composition vs inheritance: which should I choose ?

It depends on what you want to do !  
Usually you can settle this with the following tests:

**Structure**
  - Does this new class have the same interface as the other class ? 

**Behavior**
  - Does this new class need only a part of the behavior of the other class ? 
  
**Substitution**
  - Can the code use `SpecializedClass` instead of `BasicClass` without breaking down ? (the affirmative indicates inheritance)

See also: [Liskov principle](https://en.wikipedia.org/wiki/Liskov_substitution_principle).



### To sum up

Inheritance makes the child class expose *all* the interface of the mother class.
  - If this is what you actually intended, then inheritance is advised (ex. GUI)
  - If this is not what was primarily intended (ex. you only need a part of the features of the other class), composition should be preferred.
  
Composition often requires lots of wrapping/cabling which can be done:
* explicitely (see the previous example)
* implicitly using  `__getattr__` but it makes the code far less readable





### Example

Let us go back to our `Oxide` example.  
An `Oxide` **has a** formula, so it seems reasonable to implement the formula logic in a dedicated class, and to compose this class within `Oxide`.

```python
class Formula(object):
    def __init__(self, metal, nmet, nox):
        self.formula = self.compute_formula(metal, nmet, nox)
    def compute_formula(self, metal, nmet, nox):
        if nmet > 1:
            formula = "%s%iO%i"%(metal, nmet, nox)
        else:
            formula = "%sO%i"%(metal, nox)
        return formula
    
class Oxide:
    "Defines simple oxides"
      
    def __init__(self, metal, nmet, nox):
        "Provide the metal name and the number of them and oxygens"
        self.metal = str(metal)
        self.nmet = int(nmet)
        self.nox = int(nox)
        self._formula = Formula(self.metal, self.nmet, self.nox) # Composition

    @property
    def formula(self):
        return self._formula.formula
```

## Design Patterns

### Introduction

A Design Pattern is a generic re-usable solution.
  - Recognize a (design) problem/need in your software
  - Use a known (design) solution: a design pattern
  - Know the limits of this pattern (see "anti-patterns")

A design pattern is about *design*, not *implementation*, athough some design patterns are not relevant in certains languages (ex. Python with duck typing)

Most software engineers agree on good patterns ("this part should be designed this way") and bad patterns ("anti-patterns").

Design Patterns are sometimes obfuscated names for trivial things.

See also: [python patterns](https://github.com/faif/python-patterns).

### Design anti-patterns

Let us start with how *not* to design one's code.  
Here are two common design anti-patterns:
  - Golden Hammer: "I just learned how to use a hammer, now I see everything as a nail !"
  - God object: "This class is really the heart of our architecture, all other classes ultimately inherit from it."




### Types of design patterns

The design patterns can be classified as follows:
  - Creational
    - Factory
    - Singleton
    - Borg
  - Structural
    - Adapter: use one class as an API to a number of others
    - Facade: simplify an interface
    - Proxy: an object funnels operations to something else
  - Behavioral
    - Template or Framework
  - MVC: Model-View-Controller
    - Model is doing the work
    - Views (many) are displaying the result
    - Controller sends signal from Model <-> View


### Factory

**What**  
A factory is an object (or method) for creating class instances. 

**Usage**  
Sometimes you have different classes, each one implementing a specific feature. If the feature needed is only known at run-time, the factory provides an elegant way to create the right class instance.

**Limits**  
In simplest cases, a series of `if` is more readable.

**Implementation**  
In Python, everything is an object, including classes themselves. Objects have a default hashing function (the built-in `hash()` function). Therefore, classes themselves can be stored in dictionaries.

```python
from os.path import splitext
filename = input("Please choose a configuration file in the current folder")
config_parsers = {
    ".ini": IniParser, # a class
    ".yaml": YamlParser # another class
}
file_ext = splitext(filename)[-1]
parser = config_parsers[file_ext]() # instantiate the relevant class
```

### Singleton

**What**  
A singleton is a class which has *one single* instance.   

**Usage**  
This is used when a *single* class (instance) is needed by all the components of a system.
  - Example: shared data/configuration across modules.

**Limits**  
  - Subclassing is a problem with singletons
  - Python modules are intrisically singletons !

**Implementation**  

In Python, all classes have a `__new__` method which creates the instance (this is a factory !).

```python
class Singleton(object):
    @classmethod
    def __new__(cls, *args, **kwargs):
        if not hasattr(cls, "_inst"):
            cls._inst = super(Singleton, cls).__new__(*args, **kwargs)
        return cls._inst
```
**<font color="red">Exercice</font>**   
Build a singleton and subclass it

### Borg

**What**  
Like the singleton, the Borg is a "monostate pattern".  
The difference is that there can be multiple instance of a Borg, but they share the same *state* (not the same *identity*).

**Usage**  
Same as singleton: shared state/configuration 

**Limits**
  - The borgs do not have the same memory address
  
**Implementation**

In Python, all instances have their own attributes/methods stored in the `__dict__` dictionary.  
By forcing the instances to share the same `__dict__`, we implement a Borg (all instances have the same internal state, but they do not have the same memory address).

```python
class Borg(object):
    _shared_state = {}
    def __init__(self, *args, **kwargs):
        self.__dict__ = self._shared_state
```
**<font color="red">Exercice</font>**   
Build a singleton and subclass it

### Framework/Template/self delegation

**What**  
Defines the skeleton, deferring the definition/implementation of exact steps to subclasses or other cooperating classes.

**Usage**  
  - Natural way of sub-classing withing OOP
  - Super-class contains logic
  - Hook methods to be overwritten
  - Maximal re-use of logic
  
**Limits**  
  - Need to understand the logic of the framework


**Hollywood principle**: "don't call us, we'll call you !"

### Decorators

**What**  
A *decorator* modifies the *behavior* of a function/class/method, without being invasive on the code *sturcture*.  
Decorators are an example of [Aspect Oriented Programming](https://en.wikipedia.org/wiki/Aspect-oriented_programming).

**Usage**  
  - Logging the process of a function (or class method)
  - Warnings (ex. deprecation warnings)
  

**Limits**  
  - ??
  
**Implementation**  
A decorator is a function that takes a function as an input, and returns a (decorated) function as an output.  
Decorators are built-in in Python. You can define your own decorators.

```python
_warnings_cache = {}
def deprecated(func):
    def wrapper(*args, **kwargs):
        func_name = func.__name__
        if func_name not in _warnings_cache:
            _warnings_cache[func_name] = 1
            print("Warning: function %s is deprecated !" % func_name)
        return func(*args, **kwargs)
    return wrapper

@deprecated
def my_old_function(a):
    return a+1

my_old_function(1) # displays a warning
my_old_function(2)
```

